In [1]:
%matplotlib inline

In [2]:
%%capture
!pip install mne
!pip install moabb

In [3]:
import os
import numpy as np
import mne
import moabb.datasets

In [4]:
sampling_rate = 512

dataset = moabb.datasets.bi2013a(
    NonAdaptive=True,
    Adaptive=True,
    Training=True,
    Online=True,
)

In [5]:
%%capture
dataset.download(subject_list=[1, 2, 3])

SHA256 hash of downloaded file: f55a888da35e2ddcaa54f7c4848989f414636ba52b143b9a3f5a73002973b807
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 5c2905f913c0e47985ab08dc33fc022535ae9fd7132239f5d6d3a6edd803d472
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 5fc447214d0b571174044667d8bf56f6b11b4d6cd742684bd91bfadc101db836
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 9edf0a5aac449be4af78cbcb5b45e215406d3cda96b02646d87023e89a2744c2
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: e31c

In [6]:
data = dataset.get_data(subjects=[1, 2, 3])

In [7]:
runs_list = []

for subject_idx in range(1, 4):
    for session_id in data[subject_idx].keys():
        cur_session = data[subject_idx][session_id]

        for run_id in cur_session:
            cur_run = cur_session[run_id]
            runs_list.append(cur_run)

raw = mne.concatenate_raws(runs_list)

print(raw.info)

<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, Fp2, F5, AFz, F6, T7, Cz, T8, P7, P3, Pz, P4, P8, O1, Oz, ...
 chs: 16 EEG, 1 Stimulus
 custom_ref_applied: False
 dig: 19 items (3 Cardinal, 16 EEG)
 highpass: 0.0 Hz
 lowpass: 256.0 Hz
 meas_date: unspecified
 nchan: 17
 projs: []
 sfreq: 512.0 Hz
>


In [8]:
raw.filter(l_freq=1., h_freq=None)

Filtering raw data in 96 contiguous segments
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 1691 samples (3.303 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.2s finished


<RawArray | 17 x 12573024 (24556.7 s), ~1.59 GB, data loaded>

In [9]:
events = mne.find_events(raw, stim_channel='STI 014')
print(events[:10])

33768 events found
Event IDs: [33285 33286]
[[41744     0 33285]
 [42042     0 33286]
 [42147     0 33286]
 [42265     0 33286]
 [42435     0 33286]
 [42588     0 33286]
 [42854     0 33286]
 [42998     0 33285]
 [43229     0 33286]
 [43323     0 33286]]


In [10]:
event_dict = {'NonTarget': 33286, 'Target': 33285}

In [11]:
reject_criteria = dict(#mag=4000e-15,    
                       #grad=4000e-13,    
                       eeg=150e-6,     
                       #eog=250e-6
                       )  


In [12]:
%%capture
epochs = mne.Epochs(raw, events, event_id=event_dict, tmin=-0.1, tmax=0.9, decim=5,
                    preload=True, baseline=(0, 0), reject=reject_criteria)

## Export Epochs to CSV

In [13]:
epochs.to_data_frame().to_csv('./data/epochs.csv')